In [ ]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders_prince
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *

In [ ]:
plates = ["310_20230830"]
directory_targ = directory_project
# update_analysis_info(directory_targ)
# analysis_info = get_analysis_info(directory_targ)
# analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"] == "310_20230830"]
folders = folders.sort_values(by="datetime")
folders = folders.iloc[50:52]
folders

In [ ]:
# folders = all_folders.loc[all_folders["unique_id"] == "1045_20220504"]
# folders = folders.sort_values(by="datetime")
# folders = folders.iloc[60:62]

exp = Experiment(directory_targ)
exp.load(folders)
exp.dates.sort()
for t in range(len(folders)):
    exp.load_tile_information(t)

In [ ]:
edges = get_all_edges(exp, t)
edges[10].width(t)

In [ ]:
plt.close("all")
t = 0
vmax = 0
vmin = 7
ax = plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=True,
    dilation=20,
    figsize=(12, 8),
    label_colorbar="log CFBC",
)

In [ ]:
vmax = 10
vmin = 2
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=20,
    figsize=(12, 8),
)
# path = r"/scratch-shared/amftrack/transport/full_picture/"
# file_name = f'width.png'
# save_path = os.path.join(path,file_name)
# plt.savefig(save_path, dpi=dpi)

In [ ]:
t = 20
thresh = 10 ** (-6)
edges = get_all_edges(exp, t)
edges = [
    edge
    for edge in edges
    if edge.width(t) > 7 and edge.current_flow_betweeness(t) > thresh
]

In [ ]:
edges = [edge for edge in edges if len(edge.ts()) >= 10]
# edges = [edge for edge in edges if len(edge.begin.ts())==42]
# edges

In [ ]:
# edges = [edge for edge in edges if len(edge.begin.ts())==42]
plt.close("all")
vmax = 0
vmin = 1
ax = plot_edge_color_value(
    exp,
    t,
    lambda edge: edge in edges,
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=20,
    figsize=(12, 8),
    label_colorbar="log CFBC",
)

In [ ]:
fig, ax = plt.subplots()
edge_choice = np.random.choice(edges, 5)
for edge in edge_choice:
    widths = [edge.width(t) for t in edge.ts()]
    ax.plot(edge.ts(), widths)
ax2 = ax.twinx()
for edge in edge_choice:
    cenralities_m = centralities_min[edge.ts()]
    cenralities_M = centralities_max[edge.ts()]
    widths = np.log10(
        (np.array([edge.current_flow_betweeness(t) for t in edge.ts()]) - cenralities_m)
        / (cenralities_M - cenralities_m)
    )
    ax2.plot(edge.ts(), widths, linestyle="--")

In [ ]:
fig, ax = plt.subplots()
edge_choice = np.random.choice(edges, 5)
for edge in edge_choice:
    widths = [edge.width(t) for t in edge.ts()]
    ax.plot(edge.ts(), widths)
ax2 = ax.twinx()
for edge in edge_choice:
    cenralities_m = centralities_min[edge.ts()]
    cenralities_M = centralities_max[edge.ts()]
    widths = np.log10(
        (np.array([edge.betweeness(t) for t in edge.ts()]) - cenralities_m)
    )
    ax2.plot(edge.ts(), widths, linestyle="--")

In [ ]:
edgess = [get_all_edges(exp, t) for t in range(0, 41)]
centralities = [
    [edge.current_flow_betweeness(t) for edge in edges]
    for t, edges in enumerate(edgess)
]
centralities_max = np.array([np.max(centraliti) for centraliti in centralities])
centralities_min = np.array([np.min(centraliti) for centraliti in centralities])

In [ ]:
edgess = [get_all_edges(exp, t) for t in range(0, 41)]
centralities = [
    [edge.betweeness(t) for edge in edges] for t, edges in enumerate(edgess)
]
centralities_max2 = np.array([np.max(centraliti) for centraliti in centralities])
centralities_min2 = np.array([np.min(centraliti) for centraliti in centralities])

In [ ]:
widths

In [ ]:
thresh = 1e-4
widths = [
    edge.width(t)
    for edge in edges
    if edge.width(t) > 0 and edge.current_flow_betweeness(t) > thresh
]
centrality = [
    np.log10(edge.current_flow_betweeness(t))
    for edge in edges
    if edge.width(t) > 0 and edge.current_flow_betweeness(t) > thresh
]
fig, ax = plt.subplots()
ax.scatter(widths, centrality, alpha=0.2)
# ax.set_xlim(4,8)

In [ ]:
thresh = 1e-4
widths = [
    edge.width(t)
    for edge in edges
    if edge.width(t) > 0 and edge.current_flow_betweeness(t) > thresh
]
centrality = [
    np.log10(edge.betweeness(t))
    for edge in edges
    if edge.width(t) > 0 and edge.current_flow_betweeness(t) > thresh
]
fig, ax = plt.subplots()
ax.scatter(widths, centrality, alpha=0.2)
# ax.set_xlim(4,8)

In [ ]:
fig, ax = plt.subplots()
ax.hist(centrality, 20)

In [ ]:
fig, ax = plt.subplots()
ax.hist(widths, 20)